### Prompting Chaining with News 

In [2]:

#=======================#
# ---- libraries ----- #
#=======================#

from pydantic import BaseModel, Field, validator
from typing import List, Optional, Dict
from datetime import datetime, date
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import time
import re
from enum import Enum
from openai import OpenAI  

In [3]:
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [10]:
#========================#
# ---- model design ---- #
#========================#


class OpenAIModels(str, Enum):
    GPT_4O_MINI = "gpt-4o-mini"
    GPT_41_MINI = "gpt-4.1-mini"
    GPT_41_NANO = "gpt-4.1-nano"


MODEL = OpenAIModels.GPT_41_NANO
MAX_ARTICLES = 15
BASE_URL = 'https://cnnespanol.cnn.com/colombia'


In [5]:
#================================#
# ---- Helper Functions ---- #
#================================#

def get_completion(messages=None, 
                   system_prompt=None, 
                   user_prompt=None, 
                   model=MODEL):
    """Llamada a OpenAI API"""
    messages = list(messages) if messages else []
    if system_prompt:
        messages.insert(0, {"role": "system", "content": system_prompt})
    if user_prompt:
        messages.append({"role": "user", "content": user_prompt})
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.000001,
    )
    return response.choices[0].message.content


def extract_date_from_url(url: str) -> Optional[date]:
    """Extrae fecha de URL formato /YYYY/MM/DD/"""
    pattern = r'/(\d{4})/(\d{2})/(\d{2})/'
    match = re.search(pattern, url)
    
    if match:
        year, month, day = map(int, match.groups())
        try:
            return date(year, month, day)
        except ValueError:
            return None
    return None


def is_today(article_date: Optional[date]) -> bool:
    """Verifica si la fecha es HOY (dinámico)"""
    if not article_date:
        return False
    return article_date == date.today()

In [6]:
#================================#
# ---- Pydantic Models ---- #
#================================#

class ArticleSummary(BaseModel):
    """Paso 1: Resumen"""
    article_id: str
    summary: str
    key_points: List[str]
    
    @validator('summary')
    def validate_summary(cls, v):
        if len(v) < 50:
            raise ValueError('Resumen muy corto (mínimo 50 caracteres)')
        return v


class ArticleTopic(BaseModel):
    """Paso 2: Topic"""
    article_id: str
    primary_topic: str
    categories: List[str]
    
    @validator('primary_topic')
    def validate_topic(cls, v):
        if not v or len(v) < 3:
            raise ValueError('Topic principal no identificado')
        return v


class ArticleRelevance(BaseModel):
    """Paso 3: Relevancia"""
    article_id: str
    relevance_score: float
    impact_level: str
    reasoning: str
    urgency: str
    
    @validator('relevance_score')
    def validate_score(cls, v):
        if not (0 <= v <= 10):
            raise ValueError('Score debe estar entre 0 y 10')
        return v


class FinalRanking(BaseModel):
    """Paso 4: Ranking"""
    most_important_id: str
    ranking: List[Dict]
    executive_summary: str


/var/folders/11/06r7d20d0z1df3fwdm3jl16c0000gn/T/ipykernel_65255/246959497.py:11: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('summary')
/var/folders/11/06r7d20d0z1df3fwdm3jl16c0000gn/T/ipykernel_65255/246959497.py:24: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  @validator('primary_topic')
/var/folders/11/06r7d20d0z1df3fwdm3jl16c0000gn/T/ipykernel_65255/246959497.py:39: PydanticDeprecatedSince20: Pydantic V1 style

In [7]:
#================================#
# ---- Gate Check Functions ---- #
#================================#

def gate_check_date(article_dict: dict) -> tuple[bool, str]:
    """
    GATE CHECK 1: Verificar que sea de HOY
    """
    url = article_dict.get('url', '')
    article_date = extract_date_from_url(url)
    
    if not article_date:
        return False, "No se pudo extraer fecha"
    
    if not is_today(article_date):
        return False, f"No es de hoy ({article_date.strftime('%d/%m/%Y')})"
    
    return True, "✓ Fecha válida"


def gate_check_content_quality(article_dict: dict) -> tuple[bool, List[str]]:
    """
    GATE CHECK 2: Calidad del contenido
    """
    errors = []
    
    title = article_dict.get('title', '')
    if len(title) < 10:
        errors.append(f"Título muy corto: {len(title)} chars")
    
    content = article_dict.get('content', '')
    if len(content) < 50:
        errors.append(f"Contenido muy corto: {len(content)} chars")
    
    word_count = len(content.split())
    if word_count < 50:
        errors.append(f"Pocas palabras: {word_count}")
    
    if content.count('<') > 3:
        errors.append("Contiene HTML")
    
    if content.count('.') < 2:
        errors.append("Sin estructura de párrafos")
    
    return len(errors) == 0, errors



In [12]:
#========================================#
# ---- Web Scraper with Date Filter ---- #
#========================================#

print("="*80)
print("SCRAPING CNN ESPANOL - COLOMBIA")
print("="*80)
print(f"Fecha objetivo: {date.today().strftime('%A, %d de %B de %Y')}")
print(f"Maximo articulos: {MAX_ARTICLES}")
print("="*80 + "\n")

valid_articles = []
rejected_articles = []

try:
    # Descargar pagina principal
    response = requests.get(BASE_URL, timeout=10)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    
    print("Extrayendo enlaces de articulos...\n")
    
    # Extraer enlaces y FILTRAR por fecha ANTES de agregar
    article_links = []
    seen_urls = set()
    total_found = 0
    filtered_by_date = 0
    
    for link in soup.find_all('a', class_='container__link', href=True):
        href = link.get('href', '')
        
        # Construir URL completa
        if not href.startswith('http'):
            href = f"https://cnnespanol.cnn.com{href}"
        
        # GATE CHECK DE FECHA AQUI - ANTES de agregar a la lista
        article_date = extract_date_from_url(href)
        
        if not is_today(article_date):
            filtered_by_date += 1
            continue  # Saltar articulos que no son de hoy
        
        # Extraer titulo
        title_span = link.find('span', class_='container__headline-text')
        
        if title_span and href not in seen_urls:
            title = title_span.get_text(strip=True)
            article_links.append({'title': title, 'url': href})
            seen_urls.add(href)
            total_found += 1
            
            print(f"[{len(article_links)}] Encontrado: {title[:60]}...")
            
            if len(article_links) >= MAX_ARTICLES:
                break
    
    print(f"\n{'='*80}")
    print(f"Enlaces encontrados de HOY: {len(article_links)}")
    print(f"Enlaces filtrados (no son de hoy): {filtered_by_date}")
    print(f"{'='*80}\n")
    
    if not article_links:
        print("ADVERTENCIA: No se encontraron articulos de hoy.")
        print("Posibles razones:")
        print("1. No hay noticias publicadas hoy en esta seccion")
        print("2. La estructura HTML de CNN cambio")
        print("3. Problema de conexion\n")
    
    # Descargar contenido de cada articulo
    print("Descargando contenido de articulos...\n")
    
    for i, article_data in enumerate(article_links, 1):
        print(f"[{i}/{len(article_links)}] {article_data['title'][:50]}...")
        
        try:
            article_response = requests.get(article_data['url'], timeout=10)
            article_response.raise_for_status()
            article_soup = BeautifulSoup(article_response.content, 'html.parser')
            
            # Extraer parrafos
            paragraphs = article_soup.find_all('p', class_='paragraph')
            content = "\n\n".join([p.get_text(strip=True) for p in paragraphs])
            
            if not content:
                print(f"   RECHAZADO - Sin contenido")
                rejected_articles.append({
                    **article_data, 
                    'reason': 'sin_contenido',
                    'date': date.today()
                })
                time.sleep(0.3)
                continue
            
            # Agregar informacion
            article_data['content'] = content
            article_data['word_count'] = len(content.split())
            article_data['scraped_at'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            article_data['publication_date'] = date.today()
            
            # Gate Check: Calidad de contenido
            is_valid_quality, quality_errors = gate_check_content_quality(article_data)
            
            if not is_valid_quality:
                print(f"   RECHAZADO - {'; '.join(quality_errors)}")
                rejected_articles.append({
                    **article_data, 
                    'reason': 'calidad_insuficiente',
                    'errors': quality_errors
                })
                time.sleep(0.3)
                continue
            
            # Articulo VALIDO
            valid_articles.append(article_data)
            print(f"   VALIDO - {article_data['word_count']} palabras")
            
        except Exception as e:
            print(f"   ERROR - {str(e)[:50]}")
            rejected_articles.append({
                **article_data, 
                'reason': 'error_descarga',
                'error': str(e)
            })
        
        time.sleep(0.5)  # Rate limiting
    
    # Resumen final
    print("\n" + "="*80)
    print("RESUMEN DEL SCRAPING")
    print("="*80)
    print(f"Enlaces procesados: {len(article_links)}")
    print(f"Articulos validos: {len(valid_articles)}")
    print(f"Articulos rechazados: {len(rejected_articles)}")
    if article_links:
        print(f"Tasa de exito: {len(valid_articles)/len(article_links)*100:.1f}%")
    print("="*80)
    
except Exception as e:
    print(f"ERROR FATAL: {e}")

# Mostrar preview
if valid_articles:
    print(f"\nPreview de articulos validos:")
    for i, art in enumerate(valid_articles[:3], 1):
        print(f"{i}. {art['title']}")
else:
    print("\nNo se encontraron articulos validos de hoy.")

SCRAPING CNN ESPANOL - COLOMBIA
Fecha objetivo: Monday, 05 de January de 2026
Maximo articulos: 15

Extrayendo enlaces de articulos...

[1] Encontrado: "Es como un respiro, como si algo te soltara": el testimonio...
[2] Encontrado: Estos son los países a los que Trump lanzó advertencias tras...
[3] Encontrado: Petro habla de "tomar las armas" para defender la soberanía ...
[4] Encontrado: Tras los ataques de EE.UU. en Venezuela, ¿qué pueden esperar...

Enlaces encontrados de HOY: 4
Enlaces filtrados (no son de hoy): 69

Descargando contenido de articulos...

[1/4] "Es como un respiro, como si algo te soltara": el ...
   RECHAZADO - Sin contenido
[2/4] Estos son los países a los que Trump lanzó adverte...
   VALIDO - 1158 palabras
[3/4] Petro habla de "tomar las armas" para defender la ...
   VALIDO - 269 palabras
[4/4] Tras los ataques de EE.UU. en Venezuela, ¿qué pued...
   VALIDO - 1366 palabras

RESUMEN DEL SCRAPING
Enlaces procesados: 4
Articulos validos: 3
Articulos rechazados: 1


In [13]:
valid_articles

[{'title': 'Estos son los países a los que Trump lanzó advertencias tras el ataque en Venezuela',
  'url': 'https://cnnespanol.cnn.com/2026/01/05/eeuu/paises-trump-advertencias-ataque-venezuela-trax',
  'content': 'Desde que las fuerzas de Estados Unidos capturaron al presidente de Venezuela, Nicolás Maduro, durante el fin de semana, el presidente Donald Trump y miembros de su Gobierno han emitido advertencias a varios países y territorios, entre ellos Colombia, Cuba, México, Irán y Groenlandia, un territorio autónomo de Dinamarca.\n\nTrump dijo el domingo: “Nuestro objetivo es tener países a nuestro alrededor que sean viables y exitosos y donde se permita que el petróleo salga libremente”.\n\n“El dominio estadounidense en el hemisferio occidental no volverá a ser cuestionado”, afirmó Trump.\n\nEsto es lo que hay que saber sobre lo que Trump ha dicho en los últimos dos días y cómo han respondido algunos de esos Gobiernos.\n\nTrump reiteró el domingo que Estados Unidos necesita la enorm

In [14]:
#### Prompt Chaining  ####


#=======================================#
# ---- PASO 1: Resumir Noticias ---- #
#=======================================#

print("="*80)
print("PASO 1: RESUMIENDO NOTICIAS")
print("="*80 + "\n")

if not valid_articles:
    print("No hay articulos para resumir.")
else:
    summaries = {}
    
    for i, article in enumerate(valid_articles, 1):
        print(f"[{i}/{len(valid_articles)}] {article['title'][:50]}...")
        
        system_prompt = """Eres un periodista experto en sintetizar ideas de forma que verifica lo que escriba y lo respalda con la funete para asi poder dar conclusiones verdaderas y coherentes. 
        Resume la noticia en 2-3 oraciones y extrae 3-5 puntos clave."""
        
        user_prompt = f"""Resume esta noticia:

Titulo: {article['title']}
Contenido: {article['content'][:2000]}

Formato:
RESUMEN: [2-3 oraciones]
PUNTOS CLAVE:
- [punto 1]
- [punto 2]
- [punto 3]"""
        
        try:
            response = get_completion(system_prompt=system_prompt, user_prompt=user_prompt)
            
            # Parsear respuesta
            summary_match = re.search(r'RESUMEN:\s*(.+?)(?=PUNTOS CLAVE:|$)', response, re.DOTALL)
            points_section = re.search(r'PUNTOS CLAVE:\s*(.+)', response, re.DOTALL)
            
            if summary_match:
                summary_text = summary_match.group(1).strip()
                
                key_points = []
                if points_section:
                    points_text = points_section.group(1)
                    key_points = [
                        line.strip('- ').strip() 
                        for line in points_text.split('\n') 
                        if line.strip().startswith('-')
                    ]
                
                # Validar con Pydantic
                summary_obj = ArticleSummary(
                    article_id=article['url'],
                    summary=summary_text,
                    key_points=key_points
                )
                
                summaries[article['url']] = summary_obj
                print(f"   OK - {len(summary_text)} chars, {len(key_points)} puntos")
            else:
                print(f"   ERROR - No se pudo parsear respuesta")
        
        except Exception as e:
            print(f"   ERROR: {e}")
    
    print(f"\nCompletado: {len(summaries)}/{len(valid_articles)} resumenes generados")

PASO 1: RESUMIENDO NOTICIAS

[1/3] Estos son los países a los que Trump lanzó adverte...
   OK - 467 chars, 5 puntos
[2/3] Petro habla de "tomar las armas" para defender la ...
   OK - 588 chars, 5 puntos
[3/3] Tras los ataques de EE.UU. en Venezuela, ¿qué pued...
   OK - 462 chars, 5 puntos

Completado: 3/3 resumenes generados


In [15]:
summaries

{'https://cnnespanol.cnn.com/2026/01/05/eeuu/paises-trump-advertencias-ataque-venezuela-trax': ArticleSummary(article_id='https://cnnespanol.cnn.com/2026/01/05/eeuu/paises-trump-advertencias-ataque-venezuela-trax', summary='Tras la captura del presidente venezolano Nicolás Maduro, Donald Trump y su gobierno han emitido advertencias a países como Colombia, Cuba, México, Irán y Groenlandia, enfatizando la importancia de mantener la estabilidad y el dominio en el hemisferio occidental. Además, Trump expresó interés en adquirir Groenlandia por motivos de seguridad nacional, lo que generó rechazo por parte del gobierno de la isla, que calificó sus declaraciones como una falta de respeto.', key_points=['Trump advirtió a varios países del hemisferio occidental tras la captura de Maduro, reafirmando el dominio de EE.UU. en la región.', 'El presidente estadounidense expresó interés en adquirir Groenlandia por su valor estratégico y recursos, lo que fue rechazado por el gobierno de la isla.', 'L

In [16]:
#========================================#
# ---- PASO 2: Identificar Topics ---- #
#========================================#

print("="*80)
print("PASO 2: IDENTIFICANDO TOPICS")
print("="*80 + "\n")

if not summaries:
    print("No hay resumenes disponibles.")
else:
    topics = {}
    
    for i, article in enumerate(valid_articles, 1):
        if article['url'] not in summaries:
            continue
        
        print(f"[{i}/{len(valid_articles)}] {article['title'][:50]}...")
        
        summary = summaries[article['url']]
        
        system_prompt = """Eres experto en clasificacion de noticias. Vas a leer el resumen de una noticia y debes identificar su tema principal y categorizarla."""
        
        user_prompt = f"""Analiza:

Titulo: {article['title']}
Resumen: {summary.summary}

Identifica:
TEMA: [tema principal en 2-3 palabras]
CATEGORIAS: [politica, economia, sociedad, deportes, medio ambiente, geografia, religión, salud, tecnologia, cultura, relaciones internacional, gastronomia, o en un caso que no encaje podras decir no definido]"""
        
        try:
            response = get_completion(system_prompt=system_prompt, user_prompt=user_prompt)
            
            topic_match = re.search(r'TEMA:\s*(.+)', response)
            categories_match = re.search(r'CATEGORIAS:\s*(.+)', response)
            
            if topic_match:
                primary_topic = topic_match.group(1).strip()
                
                categories = []
                if categories_match:
                    categories_text = categories_match.group(1)
                    categories = [cat.strip() for cat in categories_text.split(',')]
                
                topic_obj = ArticleTopic(
                    article_id=article['url'],
                    primary_topic=primary_topic,
                    categories=categories
                )
                
                topics[article['url']] = topic_obj
                print(f"   OK - Topic: {primary_topic} | {', '.join(categories)}")
            else:
                print(f"   ERROR - No se pudo identificar topic")
        
        except Exception as e:
            print(f"   ERROR: {e}")
    
    print(f"\nCompletado: {len(topics)}/{len(summaries)} topics identificados")

PASO 2: IDENTIFICANDO TOPICS

[1/3] Estos son los países a los que Trump lanzó adverte...
   OK - Topic: Política internacional | Relaciones internacionales
[2/3] Petro habla de "tomar las armas" para defender la ...
   OK - Topic: Conflicto diplomático | Relaciones internacionales, política
[3/3] Tras los ataques de EE.UU. en Venezuela, ¿qué pued...
   OK - Topic: Conflicto regional | relaciones internacionales, política

Completado: 3/3 topics identificados


In [26]:
topics

{'https://cnnespanol.cnn.com/2026/01/05/eeuu/paises-trump-advertencias-ataque-venezuela-trax': ArticleTopic(article_id='https://cnnespanol.cnn.com/2026/01/05/eeuu/paises-trump-advertencias-ataque-venezuela-trax', primary_topic='Política internacional', categories=['Relaciones internacionales']),
 'https://cnnespanol.cnn.com/2026/01/05/colombia/petro-trump-amenaza-armas-venezuela-colombia-orix': ArticleTopic(article_id='https://cnnespanol.cnn.com/2026/01/05/colombia/petro-trump-amenaza-armas-venezuela-colombia-orix', primary_topic='Conflicto diplomático', categories=['Relaciones internacionales', 'política']),
 'https://cnnespanol.cnn.com/2026/01/05/mexico/ataques-eeuu-venezuela-colombia-mexico-orix': ArticleTopic(article_id='https://cnnespanol.cnn.com/2026/01/05/mexico/ataques-eeuu-venezuela-colombia-mexico-orix', primary_topic='Conflicto regional', categories=['relaciones internacionales', 'política'])}

In [41]:
#===========================================#
# ---- PASO 3: Evaluar Relevancia ---- #
#===========================================#

print("="*80)
print("PASO 3: CALIFICANDO RELEVANCIA")
print("="*80 + "\n")

if not topics:
    print("No hay topics disponibles.")
else:
    relevances = {}
    
    for i, article in enumerate(valid_articles, 1):
        if article['url'] not in summaries or article['url'] not in topics:
            continue
        
        print(f"[{i}/{len(valid_articles)}] {article['title'][:50]}...")
        
        summary = summaries[article['url']]
        topic = topics[article['url']]
        
        system_prompt = """Eres editor de noticias, tu misión es evaluar la relevancia de las noticias, basado en el resumen y la categoria , 
        no debe haber empate entre las noticias puedes usar ahsta 10 decimales para desempatar las noticias en su score. Califica relevancia:
- Impacto (0-4)
- Urgencia (0-3)  
- Alcance (0-3)
Recuerda no debe haber empates entre las noticias, usa decimales para diferenciarlas.
"""
        
        user_prompt = f"""Evalua:

Titulo: {article['title']}
Resumen: {summary.summary}
Tema: {topic.primary_topic}

Proporciona:
SCORE: [0.0-10.0]
IMPACTO: [Alto/Medio/Bajo]
RAZONAMIENTO: [2 oraciones]
URGENCIA: [Urgente/Importante/Normal] Recuerda no debe haber empates entre las noticias, usa decimales para diferenciarlas."""
        
        try:
            response = get_completion(system_prompt=system_prompt, user_prompt=user_prompt)
            
            score_match = re.search(r'SCORE:\s*([\d.]+)', response)
            impact_match = re.search(r'IMPACTO:\s*(\w+)', response)
            reasoning_match = re.search(r'RAZONAMIENTO:\s*(.+?)(?=URGENCIA:|$)', response, re.DOTALL)
            urgency_match = re.search(r'URGENCIA:\s*(\w+)', response)
            
            if score_match:
                score = float(score_match.group(1))
                impact = impact_match.group(1) if impact_match else "Medio"
                reasoning = reasoning_match.group(1).strip() if reasoning_match else ""
                urgency = urgency_match.group(1) if urgency_match else "Normal"
                
                relevance_obj = ArticleRelevance(
                    article_id=article['url'],
                    relevance_score=score,
                    impact_level=impact,
                    reasoning=reasoning,
                    urgency=urgency
                )
                
                relevances[article['url']] = relevance_obj
                print(f"   OK - Score: {score}/10 | {impact} | {urgency}")
            else:
                print(f"   ERROR - No se pudo extraer score")
        
        except Exception as e:
            print(f"   ERROR: {e}")
    
    print(f"\nCompletado: {len(relevances)}/{len(topics)} evaluaciones")

PASO 3: CALIFICANDO RELEVANCIA

[1/3] Estos son los países a los que Trump lanzó adverte...
   OK - Score: 8.123456789/10 | Alto | Urgente
[2/3] Petro habla de "tomar las armas" para defender la ...
   OK - Score: 8.123456789/10 | Alto | Urgente
[3/3] Tras los ataques de EE.UU. en Venezuela, ¿qué pued...
   OK - Score: 9.1245678901/10 | Alto | Urgente

Completado: 3/3 evaluaciones


In [40]:
relevances

{'https://cnnespanol.cnn.com/2026/01/05/eeuu/paises-trump-advertencias-ataque-venezuela-trax': ArticleRelevance(article_id='https://cnnespanol.cnn.com/2026/01/05/eeuu/paises-trump-advertencias-ataque-venezuela-trax', relevance_score=8.123456789, impact_level='Alto', reasoning='La noticia involucra a figuras políticas de gran influencia y afecta la estabilidad en el hemisferio occidental, lo que puede tener repercusiones internacionales significativas. Además, la mención de intereses en Groenlandia y las advertencias a varios países elevan su relevancia global.', urgency='Urgente'),
 'https://cnnespanol.cnn.com/2026/01/05/colombia/petro-trump-amenaza-armas-venezuela-colombia-orix': ArticleRelevance(article_id='https://cnnespanol.cnn.com/2026/01/05/colombia/petro-trump-amenaza-armas-venezuela-colombia-orix', relevance_score=8.123456789, impact_level='Alto', reasoning='La declaración de Petro sobre la posibilidad de tomar las armas y la escalada en el conflicto diplomático con EE.UU. repr

In [36]:
top_5_context

'1. Estos son los países a los que Trump lanzó advertencias tras (Score: 7.5/10)\n2. Petro habla de "tomar las armas" para defender la soberanía  (Score: 7.5/10)\n3. Tras los ataques de EE.UU. en Venezuela, ¿qué pueden esperar (Score: 7.5/10)'

In [42]:
#========================================#
# ---- PASO 4: Ranking Final ---- #
#========================================#

print("="*80)
print("PASO 4: RANKING FINAL")
print("="*80 + "\n")

if not relevances:
    print("No hay evaluaciones disponibles.")
else:
    # Preparar datos
    ranking_data = []
    for article in valid_articles:
        url = article['url']
        if url in summaries and url in topics and url in relevances:
            ranking_data.append({
                'url': url,
                'title': article['title'],
                'topic': topics[url].primary_topic,
                'score': relevances[url].relevance_score,
                'impact': relevances[url].impact_level,
                'urgency': relevances[url].urgency,
                'summary': summaries[url].summary
            })
    
    # Ordenar por score
    ranking_data.sort(key=lambda x: x['score'], reverse=True)
    
    # Preparar contexto
    top_5_context = "\n".join([
        f"{i+1}. {item['title'][:60]} (Score: {item['score']}/10)"
        for i, item in enumerate(ranking_data[:5])
    ])
    
    system_prompt = """Eres director editorial. Selecciona LA noticia MAS IMPORTANTE considerando impacto, urgencia y relevancia para Colombia. 
    DEbes ser critico y objetivo para ello  debes entender la noticia y el impacto segun su categoria para Colombia. Las noticias no deben tener la misma calificacion entre si, debe haber siempre uin desempate"""
    
    user_prompt = f"""Analiza estas {len(ranking_data)} noticias de HOY:

TOP 5:
{top_5_context}

Indica:
MAS IMPORTANTE: [numero 1-5]
RESUMEN EJECUTIVO: [3-4 oraciones sobre el panorama del dia]"""
    
    try:
        response = get_completion(system_prompt=system_prompt, user_prompt=user_prompt)
        
        important_match = re.search(r'MAS IMPORTANTE:\s*(\d+)', response)
        summary_match = re.search(r'RESUMEN EJECUTIVO:\s*(.+)', response, re.DOTALL)
        
        most_important_idx = int(important_match.group(1)) - 1 if important_match else 0
        executive_summary = summary_match.group(1).strip() if summary_match else "N/A"
        
        most_important_url = ranking_data[most_important_idx]['url']
        
        final_result = FinalRanking(
            most_important_id=most_important_url,
            ranking=ranking_data,
            executive_summary=executive_summary
        )
        
        # Mostrar resultados
        print("NOTICIA MAS IMPORTANTE:")
        print("-" * 80)
        most_important = ranking_data[most_important_idx]
        print(f"Titulo: {most_important['title']}")
        print(f"Score: {most_important['score']}/10")
        print(f"Topic: {most_important['topic']}")
        print(f"Urgencia: {most_important['urgency']}")
        
        print("\n" + "="*80)
        print("RANKING COMPLETO:")
        print("="*80)
        for i, item in enumerate(ranking_data, 1):
            star = "[*]" if item['url'] == most_important_url else "   "
            print(f"{star} {i}. {item['title'][:60]}")
            print(f"       {item['score']}/10 | {item['topic']} | {item['urgency']}")
        
        print("\n" + "="*80)
        print("RESUMEN EJECUTIVO:")
        print("="*80)
        print(executive_summary)
        
    except Exception as e:
        print(f"ERROR: {e}")

PASO 4: RANKING FINAL

NOTICIA MAS IMPORTANTE:
--------------------------------------------------------------------------------
Titulo: Tras los ataques de EE.UU. en Venezuela, ¿qué pueden esperar Colombia y México?
Score: 9.1245678901/10
Topic: Conflicto regional
Urgencia: Urgente

RANKING COMPLETO:
[*] 1. Tras los ataques de EE.UU. en Venezuela, ¿qué pueden esperar
       9.1245678901/10 | Conflicto regional | Urgente
    2. Estos son los países a los que Trump lanzó advertencias tras
       8.123456789/10 | Política internacional | Urgente
    3. Petro habla de "tomar las armas" para defender la soberanía 
       8.123456789/10 | Conflicto diplomático | Urgente

RESUMEN EJECUTIVO:
Hoy se destacan las tensiones internacionales en aumento, especialmente tras los ataques de EE.UU. en Venezuela, que generan incertidumbre en la región y en Colombia. La situación refleja un escenario de posible escalada en conflictos y una mayor inestabilidad política en el continente. Además, las adverte

In [43]:
if 'final_result' in locals() and final_result:
    # Crear DataFrame
    df_data = []
    for i, item in enumerate(ranking_data, 1):
        df_data.append({
            'posicion': i,
            'es_mas_importante': item['url'] == final_result.most_important_id,
            'titulo': item['title'],
            'url': item['url'],
            'topic': item['topic'],
            'score': item['score'],
            'impacto': item['impact'],
            'urgencia': item['urgency'],
            'resumen': item['summary']
        })
    
    df = pd.DataFrame(df_data)
    
    # Guardar CSV
    filename = f"noticias_colombia_{date.today().strftime('%Y%m%d')}.csv"
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    
    print(f"Guardado en: {filename}")
    print(f"Total noticias: {len(df)}")
    
    # Mostrar preview
    display(df.head())
else:
    print("No hay resultados para exportar.")

Guardado en: noticias_colombia_20260105.csv
Total noticias: 3


,posicion,es_mas_importante,titulo,url,topic,score,impacto,urgencia,resumen
0,1,True,"Tras los ataques de EE.UU. en Venezuela, ¿qué ...",https://cnnespanol.cnn.com/2026/01/05/mexico/a...,Conflicto regional,9.124568,Alto,Urgente,Tras la operación militar de EE.UU. que captur...
1,2,False,Estos son los países a los que Trump lanzó adv...,https://cnnespanol.cnn.com/2026/01/05/eeuu/pai...,Política internacional,8.123457,Alto,Urgente,Tras la captura del presidente venezolano Nico...
2,3,False,"Petro habla de ""tomar las armas"" para defender...",https://cnnespanol.cnn.com/2026/01/05/colombia...,Conflicto diplomático,8.123457,Alto,Urgente,"El presidente de Colombia, Gustavo Petro, advi..."
